## **Restricted Boltzmann Machines**
`Restricted Boltzmann` machine is an algorithm useful for dimensionality reduction, classification, regression, collaborative filtering, feature learning and topic modeling.

In [1]:
''' required Libraries '''

import numpy as np

#### **building a class RBM and defining some functions**

In [4]:
class RBM:   
  
    def __init__(self, num_visible, num_hidden):
        self.num_hidden = num_hidden
        self.num_visible = num_visible
        self.debug_print = True

        # Initialize a weight matrix, of dimensions (num_visible x num_hidden), using
        # a uniform distribution between -sqrt(6. / (num_hidden + num_visible))
        # and sqrt(6. / (num_hidden + num_visible)). One could vary the 
        # standard deviation by multiplying the interval with appropriate value.
        # Here we initialize the weights with mean 0 and standard deviation 0.1. 
        # Reference: Understanding the difficulty of training deep feedforward 
        # neural networks by Xavier Glorot and Yoshua Bengio
        np_rng = np.random.RandomState(1234)

        self.weights = np.asarray(np_rng.uniform(
                            low=-0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                            high=0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                           size=(num_visible, num_hidden)))


        # Insert weights for the bias units into the first row and first column.
        self.weights = np.insert(self.weights, 0, 0, axis = 0)
        self.weights = np.insert(self.weights, 0, 0, axis = 1)
    
    def train(self, data, max_epochs = 1000, learning_rate = 0.1):
        """
        Train the machine.
        Parameters
        ----------
        data: A matrix where each row is a training example consisting of the states of visible units.    
        """

        num_examples = data.shape[0]

        # Insert bias units of 1 into the first column.
        data = np.insert(data, 0, 1, axis = 1)

        for epoch in range(max_epochs):      

            # Clamp to the data and sample from the hidden units. 
            # (This is the "positive CD phase", aka the reality phase.)
            pos_hidden_activations = np.dot(data, self.weights)      
            pos_hidden_probs = self._logistic(pos_hidden_activations)
            pos_hidden_probs[:,0] = 1 # Fix the bias unit.
            pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
            # Note that we're using the activation *probabilities* of the hidden states, not the hidden states       
            # themselves, when computing associations. We could also use the states; see section 3 of Hinton's 
            # "A Practical Guide to Training Restricted Boltzmann Machines" for more.
            pos_associations = np.dot(data.T, pos_hidden_probs)

            # Reconstruct the visible units and sample again from the hidden units.
            # (This is the "negative CD phase", aka the daydreaming phase.)
            neg_visible_activations = np.dot(pos_hidden_states, self.weights.T)
            neg_visible_probs = self._logistic(neg_visible_activations)
            neg_visible_probs[:,0] = 1 # Fix the bias unit.
            neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
            neg_hidden_probs = self._logistic(neg_hidden_activations)
            # Note, again, that we're using the activation *probabilities* when computing associations, not the states 
            # themselves.
            neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)

            # Update weights.
            self.weights += learning_rate * ((pos_associations - neg_associations) / num_examples)

            error = np.sum((data - neg_visible_probs) ** 2)
            if self.debug_print:
                print("Epoch %s: error is %s" % (epoch, error))

    def run_visible(self, data):
        """
        Assuming the RBM has been trained (so that weights for the network have been learned),
        run the network on a set of visible units, to get a sample of the hidden units.

        Parameters
        ----------
        data: A matrix where each row consists of the states of the visible units.

        Returns
        -------
        hidden_states: A matrix where each row consists of the hidden units activated from the visible
        units in the data matrix passed in.
        """

        num_examples = data.shape[0]

        # Create a matrix, where each row is to be the hidden units (plus a bias unit)
        # sampled from a training example.
        hidden_states = np.ones((num_examples, self.num_hidden + 1))

        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis = 1)

        # Calculate the activations of the hidden units.
        hidden_activations = np.dot(data, self.weights)
        # Calculate the probabilities of turning the hidden units on.
        hidden_probs = self._logistic(hidden_activations)
        # Turn the hidden units on with their specified probabilities.
        hidden_states[:,:] = hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
        # Always fix the bias unit to 1.
        # hidden_states[:,0] = 1

        # Ignore the bias units.
        hidden_states = hidden_states[:,1:]
        return hidden_states

    
    def run_hidden(self, data):
        """
        Assuming the RBM has been trained (so that weights for the network have been learned),
        run the network on a set of hidden units, to get a sample of the visible units.
        Parameters
        ----------
        data: A matrix where each row consists of the states of the hidden units.
        Returns
        -------
        visible_states: A matrix where each row consists of the visible units activated from the hidden
        units in the data matrix passed in.
        """

        num_examples = data.shape[0]

        # Create a matrix, where each row is to be the visible units (plus a bias unit)
        # sampled from a training example.
        visible_states = np.ones((num_examples, self.num_visible + 1))

        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis = 1)

        # Calculate the activations of the visible units.
        visible_activations = np.dot(data, self.weights.T)
        # Calculate the probabilities of turning the visible units on.
        visible_probs = self._logistic(visible_activations)
        # Turn the visible units on with their specified probabilities.
        visible_states[:,:] = visible_probs > np.random.rand(num_examples, self.num_visible + 1)
        # Always fix the bias unit to 1.
        # visible_states[:,0] = 1

        # Ignore the bias units.
        visible_states = visible_states[:,1:]
        return visible_states

    def daydream(self, num_samples):
        """
        Randomly initialize the visible units once, and start running alternating Gibbs sampling steps
        (where each step consists of updating all the hidden units, and then updating all of the visible units),
        taking a sample of the visible units at each step.
        Note that we only initialize the network *once*, so these samples are correlated.
        Returns
        -------
        samples: A matrix, where each row is a sample of the visible units produced while the network was
        daydreaming.
        """

        # Create a matrix, where each row is to be a sample of of the visible units 
        # (with an extra bias unit), initialized to all ones.
        samples = np.ones((num_samples, self.num_visible + 1))

        # Take the first sample from a uniform distribution.
        samples[0,1:] = np.random.rand(self.num_visible)

        # Start the alternating Gibbs sampling.
        # Note that we keep the hidden units binary states, but leave the
        # visible units as real probabilities. See section 3 of Hinton's
        # "A Practical Guide to Training Restricted Boltzmann Machines"
        # for more on why.
        for i in range(1, num_samples):
            visible = samples[i-1,:]

            # Calculate the activations of the hidden units.
            hidden_activations = np.dot(visible, self.weights)      
            # Calculate the probabilities of turning the hidden units on.
            hidden_probs = self._logistic(hidden_activations)
            # Turn the hidden units on with their specified probabilities.
            hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
            # Always fix the bias unit to 1.
            hidden_states[0] = 1

            # Recalculate the probabilities that the visible units are on.
            visible_activations = np.dot(hidden_states, self.weights.T)
            visible_probs = self._logistic(visible_activations)
            visible_states = visible_probs > np.random.rand(self.num_visible + 1)
            samples[i,:] = visible_states

        # Ignore the bias units (the first column), since they're always set to 1.
        return samples[:,1:]        

    def _logistic(self, x): return 1.0 / (1 + np.exp(-x))

In [5]:
if __name__ == '__main__':
    r = RBM(num_visible = 6, num_hidden = 2)
    training_data = np.array([[1,1,1,0,0,0],[1,0,1,0,0,0],[1,1,1,0,0,0],[0,0,1,1,1,0], [0,0,1,1,0,0],[0,0,1,1,1,0]])
    r.train(training_data, max_epochs = 5000)
    print(r.weights)
    user = np.array([[0,0,0,1,1,0]])
    print(r.run_visible(user))

Epoch 0: error is 8.999885382679647
Epoch 1: error is 8.788489779320464
Epoch 2: error is 8.597499003211349
Epoch 3: error is 8.32607671012215
Epoch 4: error is 8.257908789035492
Epoch 5: error is 7.9426932265575
Epoch 6: error is 7.8004520015957315
Epoch 7: error is 7.719592051184902
Epoch 8: error is 7.596385530480954
Epoch 9: error is 7.665260684281836
Epoch 10: error is 7.617092087154939
Epoch 11: error is 7.1927111718063115
Epoch 12: error is 7.411188162550983
Epoch 13: error is 7.102421447649031
Epoch 14: error is 6.9376048221310205
Epoch 15: error is 6.920457549565082
Epoch 16: error is 6.656232643485128
Epoch 17: error is 6.936197761105419
Epoch 18: error is 6.542030463027676
Epoch 19: error is 6.691221372011383
Epoch 20: error is 6.734937569203829
Epoch 21: error is 6.340678203629955
Epoch 22: error is 6.455146771429742
Epoch 23: error is 6.4825569856125
Epoch 24: error is 6.276859287561441
Epoch 25: error is 6.212948009580791
Epoch 26: error is 6.360593343808204
Epoch 27: err

Epoch 267: error is 5.137737264758853
Epoch 268: error is 5.702218282745064
Epoch 269: error is 4.893698385718993
Epoch 270: error is 5.989311381103231
Epoch 271: error is 5.660805426650401
Epoch 272: error is 5.491722306909094
Epoch 273: error is 5.483934056680739
Epoch 274: error is 5.139547807928699
Epoch 275: error is 5.748145427595277
Epoch 276: error is 5.048078389508784
Epoch 277: error is 5.118858208171365
Epoch 278: error is 5.437090779321058
Epoch 279: error is 5.479380334227335
Epoch 280: error is 5.218673500428905
Epoch 281: error is 5.505447136316788
Epoch 282: error is 5.159108526770878
Epoch 283: error is 6.162946799532044
Epoch 284: error is 5.386666423518538
Epoch 285: error is 5.67352275056815
Epoch 286: error is 5.413816914185531
Epoch 287: error is 5.4696737562403355
Epoch 288: error is 4.600858130309843
Epoch 289: error is 5.501503782537773
Epoch 290: error is 4.938377667562672
Epoch 291: error is 5.656996693036706
Epoch 292: error is 5.251770424310671
Epoch 293: e

Epoch 638: error is 1.3847212156801953
Epoch 639: error is 1.3841553074006447
Epoch 640: error is 1.38360780182508
Epoch 641: error is 1.3830780291425193
Epoch 642: error is 1.382565340026065
Epoch 643: error is 1.1653769544366446
Epoch 644: error is 1.3826231417705106
Epoch 645: error is 1.3821074745552853
Epoch 646: error is 1.4613663186211598
Epoch 647: error is 1.381237677914135
Epoch 648: error is 1.3807432325987243
Epoch 649: error is 1.6761595806936043
Epoch 650: error is 1.3789463503267703
Epoch 651: error is 1.3785194446395799
Epoch 652: error is 1.3781061645648147
Epoch 653: error is 1.4601517551108487
Epoch 654: error is 1.1718162766201683
Epoch 655: error is 1.3779192958457256
Epoch 656: error is 1.377485040032618
Epoch 657: error is 1.3770649209565184
Epoch 658: error is 1.376658401412862
Epoch 659: error is 1.5365199266970455
Epoch 660: error is 1.1699167820105183
Epoch 661: error is 1.3768144499352426
Epoch 662: error is 1.3763710105875484
Epoch 663: error is 1.375942396

Epoch 1015: error is 1.3505149951644404
Epoch 1016: error is 1.350174829902894
Epoch 1017: error is 1.3498485666855038
Epoch 1018: error is 1.3495356893544765
Epoch 1019: error is 1.3492356979834685
Epoch 1020: error is 1.0799233784536906
Epoch 1021: error is 1.0732396364852972
Epoch 1022: error is 1.350742478664064
Epoch 1023: error is 1.3503918437180427
Epoch 1024: error is 2.376116918390904
Epoch 1025: error is 2.374786878510876
Epoch 1026: error is 2.246523206471351
Epoch 1027: error is 1.354113367915969
Epoch 1028: error is 1.3536302703671086
Epoch 1029: error is 1.0443073532517124
Epoch 1030: error is 1.3542623717919684
Epoch 1031: error is 1.3537725118285358
Epoch 1032: error is 1.3533012749657651
Epoch 1033: error is 1.3528480449656959
Epoch 1034: error is 1.043869673864986
Epoch 1035: error is 1.3534927347873558
Epoch 1036: error is 1.353030972088596
Epoch 1037: error is 1.35258686846912
Epoch 1038: error is 1.3521598325217137
Epoch 1039: error is 1.3517492873776424
Epoch 1040

Epoch 1355: error is 1.3485513200394308
Epoch 1356: error is 0.970847571555366
Epoch 1357: error is 1.3493046663952466
Epoch 1358: error is 0.9670856471330113
Epoch 1359: error is 0.9622244565307174
Epoch 1360: error is 1.3512694098885045
Epoch 1361: error is 0.9587952474420111
Epoch 1362: error is 0.9541763493452077
Epoch 1363: error is 1.3532722482754944
Epoch 1364: error is 0.9510482280254822
Epoch 1365: error is 0.9466522270003309
Epoch 1366: error is 1.3552944742075714
Epoch 1367: error is 1.3546474844871061
Epoch 1368: error is 1.3540237755665059
Epoch 1369: error is 1.3534226740371627
Epoch 1370: error is 1.3528435168808195
Epoch 1371: error is 1.3522856519640964
Epoch 1372: error is 1.3517484384759801
Epoch 1373: error is 1.3512312473110215
Epoch 1374: error is 0.953199726864217
Epoch 1375: error is 0.9486577777138174
Epoch 1376: error is 1.3532375281943247
Epoch 1377: error is 2.1413525688852357
Epoch 1378: error is 1.351381133142878
Epoch 1379: error is 0.952412557425301
Epoc

...

Epoch 3435: error is 0.6797987384779514
Epoch 3436: error is 0.6797776673586788
Epoch 3437: error is 0.6797567358639122
Epoch 3438: error is 0.6797359424339354
Epoch 3439: error is 0.679715285529489
Epoch 3440: error is 0.679694763631458
Epoch 3441: error is 0.6796743752405661
Epoch 3442: error is 0.6796541188770753
Epoch 3443: error is 0.679633993080492
Epoch 3444: error is 1.6076429077296495
Epoch 3445: error is 0.6794201756135333
Epoch 3446: error is 0.6794022809151117
Epoch 3447: error is 0.6793844897830082
Epoch 3448: error is 0.6793668011151817
Epoch 3449: error is 0.6793492138234088
Epoch 3450: error is 1.6050104429664325
Epoch 3451: error is 0.6791688055867964
Epoch 3452: error is 0.6791530389233406
Epoch 3453: error is 0.6791373522701105
Epoch 3454: error is 1.583262232875445
Epoch 3455: error is 0.6792482446406114
Epoch 3456: error is 0.6792313137166995
Epoch 3457: error is 0.6792144761738712
Epoch 3458: error is 0.6791977310378142
Epoch 3459: error is 0.6791810773462366
Epoc

Epoch 3799: error is 0.677093854836588
Epoch 3800: error is 0.6770768443025246
Epoch 3801: error is 0.6770599455165072
Epoch 3802: error is 0.6770431573283925
Epoch 3803: error is 0.6770264786022386
Epoch 3804: error is 0.6770099082160783
Epoch 3805: error is 1.5834531159305965
Epoch 3806: error is 0.6771659510005226
Epoch 3807: error is 0.6771478665226766
Epoch 3808: error is 1.6183812203314047
Epoch 3809: error is 0.6769269325458847
Epoch 3810: error is 0.6769110069804647
Epoch 3811: error is 1.6163301471879221
Epoch 3812: error is 0.6767163572475191
Epoch 3813: error is 0.6767022820615338
Epoch 3814: error is 0.6766882883572855
Epoch 3815: error is 0.6766743753421502
Epoch 3816: error is 0.6766605422324866
Epoch 3817: error is 0.6766467882535153
Epoch 3818: error is 0.6766331126391998
Epoch 3819: error is 0.6766195146321323
Epoch 3820: error is 0.6766059934834189
Epoch 3821: error is 0.6765925484525664
Epoch 3822: error is 0.676579178807375
Epoch 3823: error is 1.6136264321110019
Ep

Epoch 4107: error is 0.674807032898187
Epoch 4108: error is 0.6747981714164446
Epoch 4109: error is 0.6747893486741254
Epoch 4110: error is 0.6747805643595916
Epoch 4111: error is 0.6747718181640621
Epoch 4112: error is 1.597691566966115
Epoch 4113: error is 1.5957988521498099
Epoch 4114: error is 0.6749699484826096
Epoch 4115: error is 0.67495945130802
Epoch 4116: error is 0.6749490081650539
Epoch 4117: error is 0.6749386185902744
Epoch 4118: error is 0.6749282821249534
Epoch 4119: error is 0.6749179983150025
Epoch 4120: error is 0.6749077667109085
Epoch 4121: error is 0.6748975868676685
Epoch 4122: error is 0.6748874583447296
Epoch 4123: error is 0.6748773807059261
Epoch 4124: error is 0.6748673535194224
Epoch 4125: error is 0.6748573763576545
Epoch 4126: error is 0.674847448797274
Epoch 4127: error is 0.6748375704190941
Epoch 4128: error is 0.6748277408080339
Epoch 4129: error is 0.674817959553069
Epoch 4130: error is 0.6748082262471784
Epoch 4131: error is 0.674798540487295
Epoch 4

Epoch 4720: error is 0.6722749398036858
Epoch 4721: error is 0.6722710711172978
Epoch 4722: error is 0.6722672094762816
Epoch 4723: error is 0.6722633548615086
Epoch 4724: error is 1.6151367226993416
Epoch 4725: error is 0.6722865718696217
Epoch 4726: error is 0.672282527004906
Epoch 4727: error is 0.6722784905018447
Epoch 4728: error is 0.672274462334603
Epoch 4729: error is 0.672270442477224
Epoch 4730: error is 0.6722664309036288
Epoch 4731: error is 0.6722624275876196
Epoch 4732: error is 0.6722584325028812
Epoch 4733: error is 0.6722544456229852
Epoch 4734: error is 0.6722504669213901
Epoch 4735: error is 0.672246496371446
Epoch 4736: error is 0.6722425339463969
Epoch 4737: error is 0.6722385796193843
Epoch 4738: error is 0.6722346333634498
Epoch 4739: error is 0.6722306951515384
Epoch 4740: error is 0.672226764956503
Epoch 4741: error is 0.6722228427511063
Epoch 4742: error is 0.6722189285080258
Epoch 4743: error is 0.6722150221998564
Epoch 4744: error is 0.6722111237991153
Epoch

[[ 2.53282378  0.19095787  2.5239895 ]
 [ 0.57436659 -8.05840673  4.02145617]
 [-0.74559116 -5.45849554  1.44511551]
 [ 4.89262717  1.8359155   3.92824967]
 [-0.64330188  7.72916902 -3.70109017]
 [ 0.35603421  3.25570464 -7.22585068]
 [-3.96607259 -2.46956199 -2.60713337]]
[[1. 0.]]
